In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import seaborn as sns
from sklearn.cluster import DBSCAN
from Clustering_CAML import Clustering_CAML
import h5py
from DbscanLoop import DbscanLoop
from sklearn.neighbors import NearestNeighbors
import pickle
import scipy.stats as stats
import json
from numpy.fft import fft2, fftshift, ifft2
import os.path
from scipy.ndimage import gaussian_filter
from scipy import signal
from ProgressBar import printProgressBar

from SimilarityScore import getSimilarityScore,getClusterSizesAll,getSimilarityScore_ij

from FigY_Functions import PlotScatter,LoadPoints
from FigY_Functions import FilterPoints
from FigY_Functions import GetLineOfOptima,GetLineOfOptimaUnique
from FigY_Functions import GetClusterDistribution,GetClusterSizesAlongOptima
from FigY_Functions import PlotDistribution,AnalyseClusterSizes
from FigY_Functions import GetDensity,GetOverlay,GetOptimalT
from FigY_Functions import DefineCleanedLabels

from ClustersInOutCell import ClustersInOutCell
from Finder import Finder

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'sans-serif','serif':['Arial']})

In [ ]:
# Possible parameter sets
parameters_TTX = {'mainfolder'    :'ProteinData_ttx_1hr_2/',\
                  'image_filename':'AHA_2_MMStack_Pos0.ome_locs_render_driftcor_filter_render_pix0.02X6f20_X',\
                  'incell_window' :[40000,45000,40000,45000],\
                  'noise_window'  :[50000,60000,40000,50000],\
                  'datascale'     :158,
                  'algo'          :'DbscanLoop',\
                  'analysis_name' :'dataWindow_5'};

parameters_Mike = {'mainfolder'    :'MikeData/',\
                   'image_filename' :'EGFR-P1-ATTO655_cell_2_MMImages.ome_locs_render_al_linked1sigma_X',\
                   'incell_window'  :[50,60,50,60],\
                   'noise_window'   :[50,100,100,150],\
                   'algo'           :'DbscanLoop',\
                   'analysis_name'  :'dataWindow_1'};

In [ ]:
# Set Parameters
basefolder = '../../OtherData/';

#    parameterfile = 'ProteinData_ttx_1hr_2/Analysis_dataWindow_3/dataWindow_3_parameters';
#    parameterfile = 'ProteinData_ttx_1hr_2/Analysis_dataWindow_7/dataWindow_7_parameters'; 
#parameterfile = 'AnalysisDataOrganized/TTX_control_1/Output/2022_01_07__14_30_09/parameters_window'
parameterfile = 'MikeData/Analysis_dataWindow_1/dataWindow_1_parameters';    

loadSaveParameters = 'load';
loadComputeClustering = "load";

# Loading..

## Load Parameters

In [ ]:
if(loadSaveParameters=='load'):
   # parameterfile = 'MikeData/Analysis_dataWindow_1/dataWindow_1_parameters';    
        
    with open(basefolder+parameterfile+'.json') as f:
        parameters = json.load(f);
        
    if(not ('datascale' in parameters.keys())):
        parameters['datascale'] = 1;
    
elif(loadSaveParameters=='save'):
    
    parameters                   = parameters_TTX;    
        
    if(os.path.isfile(parameterfile)):
        
        with open(parameterfile) as f:
            parameters_read = json.load(f)
        
        if(parameters_read == parameters):
            print('File exists with equal parameters');
        else:
            print('File exists with different parameters. File not saved.');
    else:
        
        if not os.path.exists((basefolder+parameters['outputfolder'])):
            os.makedirs((basefolder+parameters['outputfolder']))
                
        with open(parameterfile, 'w') as fp:
            json.dump(parameters, fp, indent=4);
            
        print("Parameters saved under "+parameterfile);
else:
    print("ERROR: Choose load or save");
    
parameters['outputfolder']   = parameters['mainfolder'] + 'Analysis_'+parameters['analysis_name']+'/';    
parameters['save_name']      = parameters['outputfolder']+parameters['analysis_name'];    
parameterfile                = basefolder+parameters['save_name']+'_parameters.json';
save_name                    = basefolder + parameters['save_name'];

## Load Points

In [ ]:
if(os.path.isfile(basefolder+parameters['save_name']+"_filtered_incell.txt")):
    XC_incell  = LoadPoints(basefolder+parameters['save_name']+"_filtered_incell.txt",datascale=parameters['datascale']);    
    XC_outcell = LoadPoints(basefolder+parameters['save_name']+"_filtered_outcell.txt",datascale=parameters['datascale']);        
else:
    XC_incell  = LoadPoints(basefolder+parameters['mainfolder']+parameters['image_filename']+'_incell.txt',datascale=parameters['datascale']);    
    XC_outcell = LoadPoints(basefolder+parameters['mainfolder']+parameters['image_filename']+'_outcell.txt',datascale=parameters['datascale']);    
    
    XC_incell   = FilterPoints(XC_incell,parameters['incell_window']);
    XC_outcell  = FilterPoints(XC_outcell,parameters['outcell_window']);
    
    XC_outcell_overlay = GetOverlay(XC_incell,XC_outcell);
    
    np.savetxt(basefolder+parameters['save_name']+"_filtered_incell.txt",XC_incell,fmt="%f\t%f");   
    np.savetxt(basefolder+parameters['save_name']+"_filtered_outcell.txt",XC_outcell,fmt="%f\t%f");

In [ ]:
#XC_incell = LoadPoints(basefolder+parameters['mainfolder']+parameters['image_filename']+'_incell.txt',datascale=158);    
#XC_outcell  = LoadPoints(basefolder+parameters['mainfolder']+parameters['image_filename']+'_outcell.txt',datascale=158);    

In [ ]:
#fig,axs = plt.subplots(1,2,figsize=(12,5));
#PlotScatter(XC_incell[:,:],ax=axs[0])
#PlotScatter(XC_outcell[:,:],ax=axs[1])

## Load Clustering Results

In [ ]:
filename = basefolder+parameters['outputfolder']+"results_"+parameters['analysis_name'];
if(loadComputeClustering == "compute"):
    FD      = Finder(algo=parameters['algo']);
    labels  = FD.fit(XC_incell);    
    
    with open(filename+'_incell.pickle','wb') as handle:
        pickle.dump({'FD':FD}, handle,protocol=pickle.HIGHEST_PROTOCOL)   
    
    FD_ref      = Finder(algo=parameters['algo']);
    labels_ref  = FD_ref.fit(XC_outcell,XC_incell);    
    
    with open(filename+'_outcell.pickle','wb') as handle:
        pickle.dump({'FD_ref':FD_ref}, handle,protocol=pickle.HIGHEST_PROTOCOL)   
    
    if(os.path.exists(filename+'.pickle')):
        os.remove(filename+'.pickle');
    #with open(filename+'.pickle','wb') as handle:
    #    pickle.dump({'FD':FD,'FD_ref':FD_ref}, handle,protocol=pickle.HIGHEST_PROTOCOL)   
elif(loadComputeClustering == "load"):
    
    if(os.path.exists(filename+'.pickle')):
        with open(filename+'.pickle', 'rb') as fr:
            FD_load = pickle.load(fr);

        FD     = FD_load['FD'];
        FD_ref = FD_load['FD_ref'];
        print("Loaded Clustering results from "+filename+'.pickle');
    else:
        with open(filename+'_incell.pickle', 'rb') as fr:
            FD_load = pickle.load(fr);
        FD     = FD_load['FD'];        
        print("Loaded Clustering results from "+filename+'_incell.pickle');

        with open(filename+'_outcell.pickle', 'rb') as fr:
            FD_load = pickle.load(fr);        
        FD_ref     = FD_load['FD_ref'];
        print("Loaded Clustering results from "+filename+'_outcell.pickle');        
        
else:
    print("ERROR");

In [ ]:
phasespace_all = FD.phasespace;
phasespace_all['labels_ref'] = FD_ref.phasespace['labels']
phasespace_all['no_clusters_ref'] = FD_ref.phasespace['no_clusters'];
phasespace_all['time_ref'] = FD_ref.phasespace['time'];

#Define abbreviated phasespace 

df1     = FD.phasespace[['sigma', 'threshold','similarityScore','no_clusters']];
df1_ref = FD_ref.phasespace[['sigma', 'threshold','similarityScore','no_clusters']];

df1['similarityScore_ref']       = df1_ref['similarityScore'];
df1['similarityScore_ref_ratio'] = (df1_ref['similarityScore'])/(df1['similarityScore']);

df1['no_clusters_ref']       = df1_ref['no_clusters'];
df1['no_clusters_ref_ratio'] = (df1_ref['no_clusters'])/(df1['no_clusters']);

#Assemble all cluster sizes
df_clusterSizes     = FD.clusterInfo;#GetClusterSizesAll(FD);
df_clusterSizes_ref = FD_ref.clusterInfo;#'GetClusterSizesAll(FD_ref);

df_clusterSizes['type'] = 'incell';
df_clusterSizes_ref['type'] = 'outcell';
df_clusterSizes_all = df_clusterSizes.append(df_clusterSizes_ref, ignore_index=True);

#Analyse line of optimal sigmas
#df_opt_sim = GetLineOfOptima(df1,'similarityScore_ref','similarityScore',15);
#df_opt_cl  = GetLineOfOptima(df1,'no_clusters_ref','no_clusters',15);

df_opt_th     = GetLineOfOptima(df1,'threshold','similarityScore')
df_opt_th_ncl = GetLineOfOptima(df1,'threshold','no_clusters')

df_clusterSizes_opt     = GetClusterSizesAlongOptima(FD,df_opt_th);
df_clusterSizes_ref_opt = GetClusterSizesAlongOptima(FD_ref,df_opt_th);

df_clusterSizes_opt['type'] = 'incell';
df_clusterSizes_ref_opt['type'] = 'outcell';
df_clusterSizes_all_opt = df_clusterSizes_opt.append(df_clusterSizes_ref_opt, ignore_index=True);

# Analysis

## Plot overview Figure

In [ ]:
# Plot Overview

fig,axs = plt.subplots(1,2,figsize=(12,5));
PlotScatter(XC_incell,ax=axs[0])
PlotScatter(XC_outcell,ax=axs[1])
#np.savetxt(outputfolder_R+analysis_name+"_filtered_incell.txt",XC_incell,fmt="%f\t%f");    
axs[0].set_title('in cell,'+str(len(XC_incell))+' points');
#axs[0].set_xlim(150,158);
#axs[0].set_ylim(140,147);

axs[1].set_title('out cell, '+str(len(XC_outcell))+' points');
for ax in axs:
    ax.set_aspect('equal');

if(True): 
    axs[0].axis('off');
    axs[1].axis('off')
    
    axs[0].set_title('Inside of cell');
    axs[1].set_title('Reference -- Outside of cell');
#axs[1].set_xlim([50,60]);
#axs[1].set_ylim([50,60]);

plt.savefig(basefolder+parameters['save_name']+"_localizations_incell_vs_outcell.pdf",bbox_inches="tight");
plt.savefig(basefolder+parameters['save_name']+"_localizations_incell_vs_outcell.png",bbox_inches="tight");

## Heatmaps Similiarity Score and Number of Clusters

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(12,13));

heatmap1_data = pd.pivot_table(df1, values='similarityScore', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=axs[0,0]);
ax.invert_yaxis()
ax.set_title('similarityScore');

heatmap1_data = pd.pivot_table(df1, values='no_clusters', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=axs[0,1]);
ax.invert_yaxis()
ax.set_title('no Clusters');

if(df1_ref['similarityScore'].isnull().sum() == 0):
    heatmap1_data = pd.pivot_table(df1_ref, values='similarityScore', 
                         index=['threshold'], 
                         columns='sigma')
    ax = sns.heatmap(heatmap1_data,ax=axs[1,0]);
    ax.invert_yaxis()
    ax.set_title('similarityScore Ref');

heatmap1_data = pd.pivot_table(df1_ref, values='no_clusters', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=axs[1,1]);
ax.invert_yaxis()
ax.set_title('no Clusters Ref');

fig.tight_layout();
plt.savefig(save_name+"_PhaseSpaces.pdf",bbox_inches="tight");

## Analyse Line Of Optima of incell signal

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(14,14));

ax = axs[0,0];
ax.plot(df_opt_th['sigma'],df_opt_th['threshold'],label='similarity Score');
sns.scatterplot(x=df_opt_th['sigma'],y=df_opt_th['threshold'],hue=df_opt_th['no_clusters_ref_ratio'],s=100,ax=ax);

ax.plot(df_opt_th_ncl['sigma'],df_opt_th_ncl['threshold'],'r',label='no Clusters');
sns.scatterplot(x=df_opt_th_ncl['sigma'],y=df_opt_th['threshold'],hue=df_opt_th['no_clusters_ref_ratio'],\
                s=100,ax=ax,palette='Reds');

ax.set_xlim(np.min(df1['sigma']),np.max(df1['sigma']))
ax.set_ylim(np.min(df1['threshold']),np.max(df1['threshold']))

ax = axs[0,1];
sns.lineplot(data=df_opt_th,x='threshold',y='similarityScore_ref_ratio',ax=ax);
sns.lineplot(data=df_opt_th_ncl,x='threshold',y='similarityScore_ref_ratio',ax=ax,color='r');

ax = axs[0,2];
sns.lineplot(data=df_opt_th,x='threshold',y='no_clusters_ref_ratio',ax=ax);
sns.lineplot(data=df_opt_th_ncl,x='threshold',y='no_clusters_ref_ratio',ax=ax,color='r');

ax = axs[1,0];
sns.lineplot(data=df_opt_th,x='threshold',y='no_clusters',ax=ax);
ax.set_ylim(0,1.1*np.max(df_opt_th['no_clusters']));

ax = axs[1,1];
sns.lineplot(data=df_opt_th,x='threshold',y='no_clusters_ref',ax=ax);
ax.set_ylim(0,1.1*np.max(df_opt_th['no_clusters_ref']));

ax = axs[1,2];
sns.lineplot(data=df_opt_th,x='no_clusters',y='no_clusters_ref',ax=ax);

ax = axs[2,0];
sns.lineplot(data=df_opt_th,x='threshold',y='similarityScore',ax=ax);
ax.set_ylim(0,1.1*np.max(df_opt_th['similarityScore']));

if(df_opt_th['similarityScore_ref'].isnull().sum()==0):
    ax = axs[2,1];
    sns.lineplot(data=df_opt_th,x='threshold',y='similarityScore_ref',ax=ax);
    ax.set_ylim(0,1.1*np.max(df_opt_th['similarityScore_ref']));

    ax = axs[2,2];
    sns.lineplot(data=df_opt_th,x='similarityScore',y='similarityScore_ref',ax=ax);

plt.savefig(save_name+"_Analysis_alongLineOfOptima.pdf",bbox_inches="tight");

In [ ]:
PlotDistribution(df_clusterSizes,df_clusterSizes_ref,save_name+"_clusterDistribution_alongLineOfOptima_incell.pdf");

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(15,7));
sns.boxplot(data=df_clusterSizes_all,y='clusterSize',x='threshold',hue='type');

## Alternative analysis of along line of optima

In [ ]:
df_incell = df_clusterSizes_all[(df_clusterSizes_all['type']=='incell')];
df_outcell  = df_clusterSizes_all[(df_clusterSizes_all['type']=='outcell')];
T_,s_vs_n_,nocl_incell_,nocl_outcell_ = GetOptimalT(df_incell,df_outcell,'similarityScore');
T_,s_vs_n_,nocl_incell_,nocl_outcell_ = GetOptimalT(df_incell,df_outcell,'clusterSize');

In [ ]:
T_

In [ ]:
phasespace_all_aboveT = DefineCleanedLabels(df_clusterSizes_all,phasespace_all,criterion='similarityScore',bestRequiredRate=1.0);

In [ ]:
i_choose  = np.argmax(phasespace_all_aboveT['no_clusters']);
ps_choose = phasespace_all_aboveT.loc[i_choose,:];

In [ ]:
clusterInfo        = getClusterSizesAll(XC_incell,phasespace_all);
clusterInfo_aboveT = getClusterSizesAll(XC_incell,phasespace_all_aboveT);

In [ ]:
cli_similarityScore,similarityScore      = getSimilarityScore(XC_incell,phasespace_all_aboveT,clusterInfo_aboveT);
phasespace_all_aboveT['similarityScore'] = similarityScore;
clusterInfo_aboveT['similarityScore']    = cli_similarityScore;

In [ ]:
df_opt_th_aboveT     = GetLineOfOptima(phasespace_all_aboveT[['sigma', 'threshold','similarityScore','no_clusters']],'threshold','similarityScore');
df_opt_th_aboveT_ncl = GetLineOfOptima(phasespace_all_aboveT[['sigma', 'threshold','similarityScore','no_clusters']],'threshold','no_clusters')

## Correlation

In [ ]:
hist2dS = np.random.rand(200,200);
hist2dS = gaussian_filter(hist2dS, sigma=10); 

#hist2dS = np.histogram2d(XC_incell[:,0],XC_incell[:,1],bins=200)[0];
hist2dS = np.histogram2d(XC_outcell[:,0],XC_outcell[:,1],bins=200)[0];
hist2dS[hist2dS>1] = 1;
#hist2dS = gaussian_filter(hist2dS, sigma=10)
#hist2dS = hist2dS.T;

plt.imshow(hist2dS);

In [ ]:
corr = signal.correlate2d(hist2dS, hist2dS,boundary='wrap', mode='full')# boundary='symm'

fig, (ax1, ax2, ax3) = plt.subplots(figsize=(13, 3), ncols=3)

pos = ax1.imshow(corr,vmin=0);
fig.colorbar(pos,ax=ax1)

ax2.plot(np.sum(corr,axis=0));
ax3.plot(np.sum(corr,axis=1));

ax2.set_ylim([0,220000]);
ax3.set_ylim([0,220000]);

In [ ]:
F = fft2(hist2dS) 
F = fftshift(F)
fft_hust2dS = np.abs(F) 

fig, (ax1, ax2, ax3) = plt.subplots(figsize=(13, 3), ncols=3)

ax1.imshow(fft_hust2dS);
#fft_hust2dS = np.abs(np.fft.fft2(hist2dS));
#plt.imshow(abs(fft_hust2dS));
ax2.plot(np.sum(fft_hust2dS,axis=0))
ax3.plot(np.sum(fft_hust2dS,axis=1))

ax2.set_ylim([0,15000]);
ax3.set_ylim([0,15000]);

#hist2dS = np.histogram2d(XC_outcell[:,0],XC_outcell[:,1],bins=100);
#plt.imshow(np.imag(np.fft.fft2(hist2dS)))

In [ ]:
hW, hH = 100,50
hFreq = 5

# Mesh on the square [0,1)x[0,1)
x = np.linspace( 0, 2*hW/(2*hW +1), 2*hW+1)     # columns (Width)
y = np.linspace( 0, 2*hH/(2*hH +1), 2*hH+1)     # rows (Height)

[X,Y] = np.meshgrid(x,y)
A = np.sin(hFreq*2*np.pi*X)

plt.imshow(A, cmap = 'gray');
H,W = np.shape(A)

F = fft2(A)/(W*H)                          
F = fftshift(F)
P = np.abs(F)                            
#plt.imshow(P);

# ***** PLot
corr = signal.correlate2d(A,A,boundary='wrap', mode='full')# boundary='symm'
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(13, 3), ncols=3)

pos = ax1.imshow(corr,vmin=0);
fig.colorbar(pos,ax=ax1)

ax2.plot(np.sum(corr,axis=0));
ax3.plot(np.sum(corr,axis=1));

#ax2.set_ylim([0,220000]);
#ax3.set_ylim([0,220000]);

# Show results of postprocessing / Exploration

In [ ]:
ps_choose

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(14,7));
PlotScatter(XC_incell,ps_choose['labels'],axs[0]);
PlotScatter(XC_outcell,ps_choose['labels_ref'],axs[1]);
axs[0].set_title('th = '+str(ps_choose['threshold'])+' , sigma = '+str(ps_choose['sigma']));
plt.savefig(save_name+"_ChossenClustering.png",bbox_inches="tight");

In [ ]:
sns.pairplot(data=df_clusterSizes_all,vars=["similarityScore","clusterSize"],hue='type')

In [ ]:
df_clusterSizes_all.to_csv(save_name+'clusterSizes_all.txt',index=False)

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(13,6)); 

sns.set_theme(style="white");

sns.histplot(data=df_clusterSizes_all,x='clusterSize',hue='type',ax=axs[0],element="step", fill=False,kde=True);

axs[0].set_xlim(0,700);
axs[0].set_ylim(0,300);

axs[1].set_ylim(0,600);
axs[1].set_xlim(0,200);

sns.histplot(data=df_clusterSizes_all,x='similarityScore',hue='type',ax=axs[1],kde=True, element="step", fill=False);
#axs[1].set_xlim(0,700);

plt.savefig(save_name+"_SizeAndSimilarity.png",bbox_inches="tight");

In [ ]:
sns.scatterplot(data=df_clusterSizes_all,x='clusterSize',y='similarityScore',alpha=0.2,hue='type');
plt.xlim(0,700);

In [ ]:
df_clusterSizes_all.groupby('type').mean()

In [ ]:
idx = 100;
row = phasespace_all_aboveT.loc[idx,:];
#row = phasespace_all.loc[idx,:];

labels_incell = row['labels'];
labels_outcell  = row['labels_ref'];
fig,axs = plt.subplots(1,2,figsize=(13,8)); 
PlotScatter(XC_incell,labels_incell,ax=axs[0])
PlotScatter(XC_outcell,labels_outcell,ax=axs[1])
axs[0].set_title('th = '+str(phasespace_all.loc[idx,'threshold'])+\
                 ' , sig = '+str(phasespace_all.loc[idx,'sigma'])+' , '+\
                 str(len(np.unique(labels_incell))-1)+' clusters');

axs[1].set_title('th = '+str(phasespace_all.loc[idx,'threshold'])+\
                 ' , sig = '+str(phasespace_all.loc[idx,'sigma'])+' , '+\
                 str(len(np.unique(labels_outcell))-1)+' clusters');

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(14,6));

ax = axs[0];
heatmap1_data = pd.pivot_table(phasespace_all_aboveT, values='percent_locsIncluded', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax,vmax=1);
ax.invert_yaxis()
ax.set_title('percent_locsIncluded_aboveT');

ax = axs[1];
heatmap1_data = pd.pivot_table(phasespace_all_aboveT, values='percent_locsIncluded_ref', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax,vmax=1.);
ax.invert_yaxis()
ax.set_title('percent_locsIncluded_aboveT_ref');

In [ ]:
#Not useful -- wanted to check whether 
phasespace_all_aboveT['no_cl_in_vs_out'] = phasespace_all_aboveT['no_clusters']/phasespace_all_aboveT['no_clusters_ref'];

fig,axs = plt.subplots(1,1,figsize=(6,6));

ax = axs;
heatmap1_data = pd.pivot_table(phasespace_all_aboveT, values='no_cl_in_vs_out', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax,vmax=3);
ax.invert_yaxis()
ax.set_title('percent_locsIncluded_aboveT');

In [ ]:
phasespace_all_aboveT.columns

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(14,6));

ax = axs[0];
heatmap1_data = pd.pivot_table(phasespace_all, values='no_clusters', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax);
ax.invert_yaxis()
ax.set_title('no_clusters');

ax = axs[1];
heatmap1_data = pd.pivot_table(phasespace_all_aboveT, values='no_clusters', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax);
ax.invert_yaxis()
ax.set_title('no_clusters above T');

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(14,6));

ax = axs[0];
heatmap1_data = pd.pivot_table(phasespace_all, values='similarityScore', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax);
ax.invert_yaxis()
ax.set_title('similarityScore');

ax = axs[1];
heatmap1_data = pd.pivot_table(phasespace_all_aboveT, values='similarityScore', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax,);
ax.invert_yaxis()
ax.set_title('similarityScore above T');

In [ ]:
df_opt_th_aboveT_ncl.head()

In [ ]:
#cli_similarityScore,similarityScore,similarityScoreMatrix = getSimilarityScoreH(XC_incell,phasespace_all_aboveT,clusterInfo_aboveT);

In [ ]:
i_check = 56;
v = [];
for i in np.arange(len(phasespace_all_aboveT)):
    s1= getSimilarityScore_ij(i_check,i,phasespace_all_aboveT,clusterInfo_aboveT)
#    print(s1)
        
    if(type(s1)!= bool):
        v.append(np.sum(s1[0]));
    else:
        v.append(0);
 #       print(np.sum(s1[0]),np.sum(s1[1]));
phasespace_all_aboveT['v'] = v;
print(np.sum(v))
print(v)


print(similarityScoreMatrix[i_check,:])
print(similarityScoreMatrix[i_check,i_check])
print(np.max(abs(similarityScoreMatrix[i_check,:]-v)))
print(np.argmax(abs(similarityScoreMatrix[i_check,:]-v)))

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(14,6));

ax = axs[0];
heatmap1_data = pd.pivot_table(phasespace_all_aboveT, values='similarityScore', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax);
ax.invert_yaxis()
ax.set_title('similarityScore');

ax = axs[1];
heatmap1_data = pd.pivot_table(phasespace_all_aboveT, values='v', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=ax);
ax.invert_yaxis()
ax.set_title('v');

In [ ]:
fix,axs=plt.subplots(2,2,figsize=(10,10))

axs[0,0].plot(df_opt_th_aboveT_ncl['threshold'],df_opt_th_aboveT_ncl['no_clusters'])
axs[0,1].plot(df_opt_th_ncl['threshold'],df_opt_th_ncl['no_clusters'])

axs[1,0].plot(df_opt_th_aboveT_ncl['threshold'],df_opt_th_aboveT_ncl['similarityScore'])
axs[1,1].plot(df_opt_th_ncl['threshold'],df_opt_th_ncl['similarityScore'])

axs[0,0].set_ylim(0,8.5);
axs[0,1].set_ylim(0,700);

axs[1,0].set_ylim(0,550);
axs[1,1].set_ylim(0,7000);

In [ ]:
threshold = 13;

df_incell = df_clusterSizes_all_opt[(df_clusterSizes_all_opt['threshold']==threshold)*\
                (df_clusterSizes_all_opt['type']=='incell')];
df_outcell  = df_clusterSizes_all_opt[(df_clusterSizes_all_opt['threshold']==threshold)*\
                (df_clusterSizes_all_opt['type']=='outcell')];
GetOptimalT(df_incell,df_outcell,criterion='similarityScore')

In [ ]:
df_opt_th_aboveT_ncl

In [ ]:
def PlotAllClustersAlongOptimum(df_opt,phasespace,name):
    for idx,row in df_opt.iterrows():
        row_ps = phasespace.loc[row['idx'],:];

        sigma = row['sigma'];
        threshold = row['threshold'];

        fig,axs = plt.subplots(1,2,figsize=(13,8)); 
        PlotScatter(XC_incell,row_ps['labels'],ax=axs[0])
        PlotScatter(XC_outcell,row_ps['labels_ref'],ax=axs[1])
        if('T' in phasespace.columns):
            axs[0].set_title('in cell for clusters with size > '+str(row_ps['T'])+' , '+str(len(np.unique(row_ps['labels']))-1)+' clusters');
            axs[1].set_title('out cell for clusters with size > '+str(row_ps['T'])+' , '+str(len(np.unique(row_ps['labels_ref']))-1)+' clusters');
        else:
            axs[0].set_title('in cell for '+str(len(np.unique(row_ps['labels']))-1)+' clusters');
            axs[1].set_title('out cell for ' +str(len(np.unique(row_ps['labels_ref']))-1)+' clusters');            

        #axs[1].set_ylim([57500,62500]);
        #axs[1].set_xlim([55000,60000]);

        if(False): 
            axs[0].axis('off');
            axs[1].axis('off')

            axs[0].set_title('Inside of cell');
            axs[1].set_title('Reference -- Outside of cell');

        plt.savefig(save_name+"_OptimalClustering_split_threshold_"+str(threshold)+"_"+name+".png",bbox_inches="tight");

In [ ]:
PlotAllClustersAlongOptimum(df_opt_th_aboveT_ncl,phasespace_all_aboveT,'aboveT');
PlotAllClustersAlongOptimum(df_opt_th_ncl,phasespace_all,'');

In [ ]:
sns.displot(data=df_clusterSizes_all_opt[df_clusterSizes_all_opt['threshold']==25], x="clusterSize",hue='type')

In [ ]:
from scipy.optimize import curve_fit
from scipy.special import factorial
from scipy.stats import poisson
from scipy.stats import nbinom

def fit_function(k, lamb):
    '''poisson function, parameter lamb is the fit parameter'''
    return poisson.pmf(k, lamb)
    
def fit_function_nb(k,n,p):
    '''poisson function, parameter lamb is the fit parameter'''
    return nbinom.pmf(k, n,p)
    
    
def GetPoissonParameters(data):
    # the bins should be of integer width, because poisson is an integer distribution
    bins = np.round(np.linspace(0,100,20));#np.arange(100) - 0.5
    entries, bin_edges, patches = plt.hist(data, bins=bins, density=True, label='Data');

    # calculate bin centres
    bin_middles = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    # fit with curve_fit
    parameters, cov_matrix = curve_fit(fit_function_nb, bin_middles, entries)
    return parameters;

In [ ]:
x_plot = np.arange(600)
np.sum(x_plot*fit_function_nb(x_plot,r,1-p))


In [ ]:
data_max = 200;

mark = (df_clusterSizes_all['threshold']==11)*(df_clusterSizes_all['type']=='noise')
data = np.array(df_clusterSizes_all.loc[mark,'clusterSize'])

data = data[data < data_max]

E,V = np.mean(data),np.var(data);
r = E**2/(V-E);
p = 1-E/V;

print('r :',r,' p: ',p);

# plot poisson-deviation with fitted parameter
bins = np.round(np.linspace(1,data_max,20));#np.arange(100) - 0.5
x_plot = np.round(np.linspace(1,data_max,20));#np.arange(100);

plt.hist(data, bins=bins, density=True, label='Data');
plt.plot(
    x_plot,
    fit_function_nb(x_plot,r,1-p),
    marker='o', linestyle='',
    label='Fit result',
)
plt.legend()
plt.show()

In [ ]:
# the bins should be of integer width, because poisson is an integer distribution
bins = np.round(np.linspace(1,100,20));#np.arange(100) - 0.5
entries, bin_edges, patches = plt.hist(data, bins=bins, density=True, label='Data');

# calculate bin centres
bin_middles = np.round(0.5 * (bin_edges[1:] + bin_edges[:-1]))

# fit with curve_fit
parameters, cov_matrix = curve_fit(fit_function_nb, bin_middles, entries)

In [ ]:
cov_matrix

In [ ]:
data

bins = np.linspace(0,100,20);
entries, bin_edges, patches = plt.hist(data, bins=bins, density=True);



In [ ]:
# get poisson deviated random numbers
#data = np.random.poisson(2, 1000)
mark = (df_clusterSizes_all['threshold']==11)*(df_clusterSizes_all['type']=='incell')
data = np.array(df_clusterSizes_all.loc[mark,'clusterSize'])
parameters = GetPoissonParameters(data)
print(parameters)

# plot poisson-deviation with fitted parameter
x_plot = np.round(np.linspace(1,100,20));#np.arange(100);

plt.plot(
    x_plot,
    fit_function_nb(x_plot,10,0.3),
    marker='o', linestyle='',
    label='Fit result',
)
plt.legend()
plt.show()

In [ ]:
df_clusterSizes_all.query('clusterSize < 100')

In [ ]:
df_clusterSizes_all.query('clusterSize < 100').groupby(by=['threshold','type']).agg(['mean','std'])

In [ ]:
df_clusterSizes_all

In [ ]:
(df_clusterSizes['type']).unique()

In [ ]:
df_stats_per_th = AnalyseClusterSizes(df_clusterSizes,df_clusterSizes_ref,save_name+"_phasespace_alongLineOfOptima.pdf");

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,6))
sns.lineplot(data=df_stats_per_th,x='threshold',y='quantile_90',hue='type',ax=axs[0]);
sns.lineplot(data=df_stats_per_th,x='threshold',y='n',hue='type',ax=axs[1]);

In [ ]:
for th in np.unique(df_opt_th['threshold']):
    idx_ = int(df_opt_th.loc[df_opt_th['threshold']==th,'idx'])

    fig,axs = plt.subplots(1,2,figsize=(13,8)); 
    PlotScatter(XC_incell,FD.phasespace.loc[idx_,'labels'],ax=axs[0])
    PlotScatter(XC_outcell,FD_ref.phasespace.loc[idx_,'labels'],ax=axs[1])
    axs[0].set_title('in cell');
    axs[1].set_title('out cell');
    
    #axs[1].set_ylim([50,70]);
    #axs[1].set_xlim([50,70]);
    
    if(True): 
        axs[0].axis('off');
        axs[1].axis('off')
    
        axs[0].set_title('Inside of cell');
        axs[1].set_title('Reference -- Outside of cell');
    
    plt.savefig(save_name+"_OptimalClustering_threshold_"+str(th)+".png",bbox_inches="tight");

## Analyse signal vs noise

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(15,7));

ax      = axs[0];
sns.scatterplot(data=df1,x='similarityScore_ref',y="similarityScore",hue='sigma',size='threshold',ax=ax);
#ax.scatter(df1.loc[idx_similarityScore,'similarityScore_ref'],df1.loc[idx_similarityScore,'similarityScore'],c='r',marker='s');
#ax.scatter(df1.loc[idx_no_clusters,'similarityScore_ref'],df1.loc[idx_no_clusters,'similarityScore'],c='b',marker='s');
sns.lineplot(data=df_opt_sim,x='similarityScore_ref',y="similarityScore",color='r',ax=ax)
sns.lineplot(data=df_opt_cl,x='similarityScore_ref',y="similarityScore",color='b',ax=ax)
ax.set_title('ordered by similarity score');

ax      = axs[1];
sns.scatterplot(data=df1,x='no_clusters_ref',y="no_clusters",hue='sigma',size='threshold',ax=ax);
#ax.scatter(df1.loc[idx_similarityScore,'no_clusters_ref'],df1.loc[idx_similarityScore,'no_clusters'],c='r',marker='s');
#ax.scatter(df1.loc[idx_no_clusters,'no_clusters_ref'],df1.loc[idx_no_clusters,'no_clusters'],c='b',marker='s');
sns.lineplot(data=df_opt_sim,x='no_clusters_ref',y="no_clusters",ax=ax,color='r')
sns.lineplot(data=df_opt_cl,x='no_clusters_ref',y="no_clusters",ax=ax,color='b')
ax.set_title('ordered by no of clusters');

plt.savefig(save_name+"_phasespace_ordered.pdf",bbox_inches="tight");
#ax.set_xlim(-10,2000);
#ax.set_xlim(0,30);

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(15,7));

ax      = axs[0];
sns.scatterplot(data=df1,x='similarityScore_ref_ratio',y="similarityScore",hue='sigma',size='threshold',ax=ax);
#ax.scatter(df1.loc[idx_similarityScore,'similarityScore_ref_ratio'],df1.loc[idx_similarityScore,'similarityScore'],c='r',marker='s');
#ax.scatter(df1.loc[idx_no_clusters,'similarityScore_ref_ratio'],df1.loc[idx_no_clusters,'similarityScore'],c='b',marker='s');
ax.set_xlim(0,1);
#ax.set_ylim(-10,4000);

ax = axs[1];
sns.scatterplot(data=df1,x='no_clusters_ref_ratio',y="no_clusters",hue='sigma',size='threshold',ax=ax);
#ax.scatter(df1.loc[idx_similarityScore,'no_clusters_ref_ratio'],df1.loc[idx_similarityScore,'no_clusters'],c='r',marker='s');
#ax.scatter(df1.loc[idx_no_clusters,'no_clusters_ref_ratio'],df1.loc[idx_no_clusters,'no_clusters'],c='b',marker='s');
ax.set_xlim(0,0.5);

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(15,7));

ax      = axs[0];
sns.scatterplot(data=df1,x='no_clusters_ref',y="similarityScore",hue='sigma',size='threshold',ax=ax);
#ax.scatter(df1.loc[idx_similarityScore,'no_clusters_ref'],df1.loc[idx_similarityScore,'similarityScore'],c='r',marker='s');
#ax.scatter(df1.loc[idx_no_clusters,'no_clusters_ref'],df1.loc[idx_no_clusters,'similarityScore'],c='b',marker='s');
#ax.set_xlim(-10,2000);
#ax.set_ylim(-10,4000);

ax = axs[1];
sns.scatterplot(data=df1,x='no_clusters_ref',y="no_clusters",hue='sigma',size='threshold',ax=ax);
#ax.scatter(df1.loc[idx_similarityScore,'no_clusters_ref'],df1.loc[idx_similarityScore,'no_clusters'],c='r',marker='s');
#ax.scatter(df1.loc[idx_no_clusters,'no_clusters_ref'],df1.loc[idx_no_clusters,'no_clusters'],c='b',marker='s');
#ax.set_xlim(-1,20);
#ax.set_ylim(-1,50);

# Save for input in R 

In [ ]:
#Save phasespace in csv
FD_all1 = (FD.phasespace);
FD_all1['type'] = 'incell';

FD_all2         = (FD_ref.phasespace);
FD_all2['type'] = 'outcell';

(FD_all1.append(FD_all2,ignore_index=True)).to_csv(outputfolder_R+'results_'+analysis_name+'.csv');
#(FD.phasespace).to_csv(outputfolder+'results_phasespace'+analysis_name+'.csv');

In [ ]:
L = np.zeros((len((FD.phasespace).loc[1,'labels']),len(FD.phasespace)),dtype=np.int32)
for i,d in enumerate(FD.phasespace['labels']):
    L[:,i] = d;
np.savetxt(outputfolder_R+analysis_name+"_labels_incell.txt",L,fmt="%d",delimiter='\t',newline='\n');    

In [ ]:
L = np.zeros((len((FD_ref.phasespace).loc[1,'labels']),len(FD_ref.phasespace)),dtype=np.int32)
for i,d in enumerate(FD_ref.phasespace['labels']):
    L[:,i] = d;
np.savetxt(outputfolder_R+analysis_name+"_labels_outcell.txt",L,fmt="%d",delimiter='\t',newline='\n');    

In [ ]:
np.savetxt(outputfolder_R+analysis_name+"_filtered_outcell.txt",XC_outcell,fmt="%f\t%f");
np.savetxt(outputfolder_R+analysis_name+"_filtered_incell.txt",XC_incell,fmt="%f\t%f");    

In [ ]:
cls_distribution = [];
ind = np.zeros((0,),dtype=int);
for i,d in enumerate(FD.phasespace['labels']):
    d_ = GetClusterDistribution(d);
    cls_distribution += (d_);
    ind = np.concatenate((ind,i*np.ones_like(d_,dtype=int)))
    
cls_d = pd.DataFrame();
cls_d['size'] = cls_distribution
cls_d['index'] = (ind);
cls_d.to_csv(outputfolder_R+analysis_name+"_clusterSizes.txt")

In [ ]:
#with open(outputfolder_R+analysis_name+"_clusterSizes.txt", "w") as txt_file:
#    for i,d in enumerate(FD.phasespace['labels']):
#        d_ = GetClusterDistribution(d);
#        txt_file.write(' '.join(str(x) for x in d_)+'\n') # works with any number of el

# Analysis

In [ ]:
#Select a clustering:
if(False):
    limit_outcell_to_incell = 0.4;
    
    # (1) based on number of clusters
    mark_ = (df1['similarityScore_ref_ratio'] < limit_outcell_to_incell);
    idx_similarityScore  = (df1.loc[mark_,'similarityScore']).idxmax();
#    print(df1.loc[idx_similarityScore,:])

    # (1) based on number of clusters
    mark_ = (df1['no_clusters_ref_ratio'] < limit_outcell_to_incell);
    idx_no_clusters  = (df1.loc[mark_,'no_clusters']).idxmax();
#    print(df1.loc[idx_no_clusters,:])
else:
    limit_outcell_no_cluster = 4;    
    limit_outcell_similarity = 320;
        
    # (1) based on number of clusters
    mark_ = (df1['similarityScore_ref'] < limit_outcell_similarity);
    idx_similarityScore  = (df1.loc[mark_,'similarityScore']).idxmax();
#    print(df1.loc[idx_similarityScore,:])

    # (1) based on number of clusters
    mark_ = (df1['no_clusters_ref'] < limit_outcell_no_cluster);
    idx_no_clusters  = (df1.loc[mark_,'no_clusters']).idxmax();
#    print(df1.loc[idx_no_clusters,:])

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(13,13)); 
PlotScatter(XC_incell,FD.phasespace.loc[idx_similarityScore,'labels'],ax=axs[0,0])
PlotScatter(XC_incell,FD.phasespace.loc[idx_no_clusters,'labels'],ax=axs[0,1])

PlotScatter(XC_outcell,FD_ref.phasespace.loc[idx_similarityScore,'labels'],ax=axs[1,0])
PlotScatter(XC_outcell,FD_ref.phasespace.loc[idx_no_clusters,'labels'],ax=axs[1,1])

In [ ]:
# Analysis: Did we reach a plateau?
def Plateau_analysis(x,y):    
    x =np.asarray(x);
    y =np.asarray(y);    
    
    y = 0.5*(y[1:]+y[:-1]);
    x = 0.5*(x[1:]+x[:-1]);    
    
    dydx = (y[1:]-y[:-1])/(x[1:]-x[:-1]);
#    print(dydx)
#    print(y)
    plt.plot(x[1:],dydx/y[0],'r');
#    plt.plot(x,y,'b');

In [ ]:
Plateau_analysis(df_stats_per_th['threshold'],df_stats_per_th['firstBin'])
plt.xlim(0,20)

In [ ]:
z = df_stats_per_th['kurtosis'];
sns.lineplot(df_stats_per_th['threshold'][1:],np.abs(np.asarray(z[1:])-np.asarray(z[:-1])),label='kurtosis');

In [ ]:
np.arange(np.max(df_clusterSizes['threshold']))

In [ ]:
sns.histplot(df_clusterSizes.loc[df_clusterSizes['threshold']==11,'clusterSize'],bins=np.linspace(0,100,101)+0.5)

In [ ]:
plt.plot(df_opt_sim['sigma'],df_opt_sim['threshold']);
sns.scatterplot(x=df_opt_sim['sigma'],y=df_opt_sim['threshold'],hue=df_opt_sim['similarityScore'],s=100);
plt.xlim(np.min(df1['sigma']),np.max(df1['sigma']))
plt.ylim(np.min(df1['threshold']),np.max(df1['threshold']))

In [ ]:
plt.plot(df_opt_cl['sigma'],df_opt_cl['threshold']);
sns.scatterplot(x=df_opt_cl['sigma'],y=df_opt_cl['threshold'],hue=df_opt_cl['no_clusters'],s=100);
plt.xlim(np.min(df1['sigma']),np.max(df1['sigma']))
plt.ylim(np.min(df1['threshold']),np.max(df1['threshold']))

In [ ]:
df1['no_clusters_diff'] = np.asarray(df1.loc[:,'no_clusters'])-np.asarray(df1_ref.loc[:,'no_clusters'])
df1 = df1.fillna(0)

fig,axs = plt.subplots(1,2,figsize=(12,5));

heatmap1_data = pd.pivot_table(df1, values='no_clusters_diff', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=axs[0]);
ax.invert_yaxis()

heatmap1_data = pd.pivot_table(df1, values='no_clusters', 
                     index=['threshold'], 
                     columns='sigma')
ax = sns.heatmap(heatmap1_data,ax=axs[1]);
ax.invert_yaxis()

In [ ]:
for th in np.unique(FD.phasespace['threshold']):
    mark = (FD.phasespace['threshold'] == th);
    PS_sel = FD.phasespace.loc[mark,:];
    idx_max = PS_sel['similarityScore'].idxmax();
    PlotScatter(XC_incell,PS_sel.loc[idx_max,'labels']);#,str(th));

In [ ]:
PS_sel.loc[:,'labels']

# Experiment

In [ ]:
# calculate the kl divergence
def kl_divergence(p, q):
    return sum(p[i] * np.log2(p[i]/q[i]) for i in range(len(p)))

In [ ]:
def GetCLDist(th):
    data = df_clusterSizes[df_clusterSizes['threshold']==th].loc[:,'clusterSize'];
    bin_no = (np.histogram(data, bins)[0])/np.sum(np.histogram(data, bins)[0]);
    return bin_no;

In [ ]:
GetCLDist(3)

In [ ]:
bins = np.linspace(0,50,11)

In [ ]:
bins

In [ ]:
np.histogram(data, bins,weights=data)[0]

In [ ]:
kl_divergence(GetCLDist(3),GetCLDist(9))

In [ ]:
np.log2(2)

In [ ]:
np.sum(mark)